In [41]:
import bs4 as bs
import urllib.request
import nltk
nltk.download("stopwords")
import re
from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to C:\Users\Latika
[nltk_data]     Bisht\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
#gettinng the data
source= urllib.request.urlopen('https://en.wikipedia.org/wiki/Global_warming').read()
print(source)

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Global warming - Wikipedia</title>\n<script>document.documentElement.className=document.documentElement.className.replace(/(^|\\s)client-nojs(\\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Global_warming","wgTitle":"Global warming","wgCurRevisionId":903638249,"wgRevisionId":903638249,"wgArticleId":5042951,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles containing potentially dated statements from May 2019","All articles containing potentially dated statements","Wikipedia articles needing page number citations from May 2019","CS1 Dutch-language sources (nl)","Webarchive template wayback links","All articles lacking reliable references","Articles lacking reliable references from June 2019","CS1: Julian\xe2\x80\x93Gregorian uncertainty"

In [48]:
#now we need to parse this complete document using beautifulsoup bcz we need string data out of it we cant use HTML data
soup= bs.BeautifulSoup(source,'lxml')

In [49]:
#now, we are going to get string data out of it
text="" #going to contain complete wikipedia article
for paragraph in soup.find_all('p'): #p is paragraph tag in HTML
    text+= paragraph.text

In [78]:
text= re.sub(r'\[[0-9]*\]',' ',text) #finding numbers in b/w the sqaure brackts and replace that pattern with a single space
text= re.sub(r'\s+',' ',text)
#we will the summary on this text bcz there will be some numbers that have to be inside of a text eg. temperature of a place is 40degree
#in case of global warming so those numbers are important 
text= text.lower()
text= re.sub(r'[@#\$%&\*\(\)\<\>\?\'\":,.;\]\[\[-]',' ',text) #conventational pre-processing
text= re.sub(r'\d',' ',text)
text= re.sub(r'\s+',' ',text)
#now, we have got the plain text so we are ready to build the model
#we will build the histogram out of this clean_text

In [79]:
sentences= nltk.sent_tokenize(text)

In [80]:
print(sentences)

[' global warming is a long term rise in the average temperature of the earth s climate system an aspect of climate change shown by temperature measurements and by multiple effects of the warming though earlier geological periods also experienced episodes of warming the term commonly refers to the observed and continuing increase in average air and ocean temperatures since caused mainly by emissions of greenhouse gases ghgs in the modern industrial economy in the modern context the terms global warming and climate change are commonly used interchangeably but climate change includes both global warming and its effects such as changes to precipitation and impacts that differ by region many of the observed changes in climate since the s are unprecedented in the instrumental temperature record and in historical and paleoclimate proxy records of climate change over thousands to millions of years in the intergovernmental panel on climate change ipcc fifth assessment report concluded it is ex

In [81]:
sentences = [nltk.word_tokenize(sentence) for sentence in sentences]

In [82]:
for i in range(len(sentences)):
    sentences[i]= [word for word in sentences[i] if word not in stopwords.words('english')]
print(sentences)

[['global', 'warming', 'long', 'term', 'rise', 'average', 'temperature', 'earth', 'climate', 'system', 'aspect', 'climate', 'change', 'shown', 'temperature', 'measurements', 'multiple', 'effects', 'warming', 'though', 'earlier', 'geological', 'periods', 'also', 'experienced', 'episodes', 'warming', 'term', 'commonly', 'refers', 'observed', 'continuing', 'increase', 'average', 'air', 'ocean', 'temperatures', 'since', 'caused', 'mainly', 'emissions', 'greenhouse', 'gases', 'ghgs', 'modern', 'industrial', 'economy', 'modern', 'context', 'terms', 'global', 'warming', 'climate', 'change', 'commonly', 'used', 'interchangeably', 'climate', 'change', 'includes', 'global', 'warming', 'effects', 'changes', 'precipitation', 'impacts', 'differ', 'region', 'many', 'observed', 'changes', 'climate', 'since', 'unprecedented', 'instrumental', 'temperature', 'record', 'historical', 'paleoclimate', 'proxy', 'records', 'climate', 'change', 'thousands', 'millions', 'years', 'intergovernmental', 'panel', 'c

# Traning word2vec model

In [83]:
model= Word2Vec(sentences, min_count=1) 
#min count means we will ignore all the words with total frequency lower than the min count so ignore all the words that appear 
#less than one time

In [84]:
words= model.wv.vocab

In [85]:
print(words)

{'global': <gensim.models.keyedvectors.Vocab object at 0x000001FAF0C44CF8>, 'warming': <gensim.models.keyedvectors.Vocab object at 0x000001FAF0C44D30>, 'long': <gensim.models.keyedvectors.Vocab object at 0x000001FAF0C44EF0>, 'term': <gensim.models.keyedvectors.Vocab object at 0x000001FAF0C44748>, 'rise': <gensim.models.keyedvectors.Vocab object at 0x000001FAF0C447B8>, 'average': <gensim.models.keyedvectors.Vocab object at 0x000001FAF0C91D68>, 'temperature': <gensim.models.keyedvectors.Vocab object at 0x000001FAF0C91B70>, 'earth': <gensim.models.keyedvectors.Vocab object at 0x000001FAF0C91F28>, 'climate': <gensim.models.keyedvectors.Vocab object at 0x000001FAF0C91C88>, 'system': <gensim.models.keyedvectors.Vocab object at 0x000001FAF0F160F0>, 'aspect': <gensim.models.keyedvectors.Vocab object at 0x000001FAF0F16128>, 'change': <gensim.models.keyedvectors.Vocab object at 0x000001FAF0F161D0>, 'shown': <gensim.models.keyedvectors.Vocab object at 0x000001FAF0F16358>, 'measurements': <gensim.

# Testing model performance

In [86]:
#lets find out the vector representation of the specific words
vector= model.wv['global']
#here we are trying to figure out what is the vector for the word global

In [87]:
#finding out the most similar word for the word warming
similar= model.wv.most_similar('warming')
print(similar)
#words trhat has appeared wid warming in most of the different sentences 

[('greenhouse', 0.37219375371932983), ('adaptation', 0.3431233763694763), ('global', 0.33356451988220215), ('solar', 0.3231053948402405), ('rest', 0.32282254099845886), ('climate', 0.3212091624736786), ('since', 0.3112810254096985), ('emissions', 0.3093642592430115), ('tipping', 0.29903921484947205), ('°c', 0.29789096117019653)]


In [88]:
similar= model.wv.most_similar('global')
print(similar)

[('earth', 0.3600684404373169), ('substantial', 0.346973717212677), ('warming', 0.33356449007987976), ('due', 0.3160253167152405), ('adapt', 0.3139323592185974), ('dioxide', 0.3085510730743408), ('climate', 0.3084118366241455), ('millions', 0.3026896119117737), ('fossil', 0.298602432012558), ('properties', 0.29748424887657166)]
